<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Downsampling</a></span></li><li><span><a href="#Очистка-признаков" data-toc-modified-id="Очистка-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Очистка признаков</a></span></li><li><span><a href="#Токенизация-текстов" data-toc-modified-id="Токенизация-текстов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Токенизация текстов</a></span></li><li><span><a href="#Создание-эмбеддингов" data-toc-modified-id="Создание-эмбеддингов-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Создание эмбеддингов</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Разделение на выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Baseline" data-toc-modified-id="Baseline-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Baseline</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#LightGBMClassifier" data-toc-modified-id="LightGBMClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LightGBMClassifier</a></span></li></ul></li><li><span><a href="#Анализ-результатов" data-toc-modified-id="Анализ-результатов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ результатов</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию человеку. 

Наша задача - обучить модель классифицировать комментарии на позитивные и негативные. 

**Описание данных**
* В нашем распоряжении набор текстов с разметкой по их токсичности.
* Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import warnings

import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import re
import requests
import torch
import transformers
import xgboost as xgb

from io import BytesIO
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_val_score,
    cross_validate
)
from sklearn.utils import shuffle
from tqdm import notebook
from transformers import AutoModel
from transformers import BertTokenizer
from transformers import pipeline
# from wordcloud import WordCloud, STOPWORDS

warnings.filterwarnings("ignore")

RS = 17

## Подготовка

### Знакомство с данными

In [2]:
try:
    df = pd.read_csv('datasets/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

df.shape

(159571, 2)

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [5]:
df['text'].apply(len).describe()

count    159571.000000
mean        394.073221
std         590.720282
min           6.000000
25%          96.000000
50%         205.000000
75%         435.000000
max        5000.000000
Name: text, dtype: float64

В дальнейшем мы планируем использовать BERT для токенизации и создания эмбеддингов, поэтому сразу удалим тексты, длинной более 512 символов.

In [6]:
X = df.loc[df['text'].str.len()<=512, 'text']
y = df.loc[df['text'].str.len()<=512, 'toxic']

In [7]:
X.shape, y.shape

((126535,), (126535,))

In [8]:
y.value_counts(normalize=True)

0    0.887841
1    0.112159
Name: toxic, dtype: float64

#### Вывод

1. В наших данных представлено 159571 наблюдений, каждое из которых имеет текст (комментарий с сайта магазина "Викишоп") и целевую метку, описывающую токсичность текста (1 - токсичный, 0 - нетоксичный).
2. Средняя длина текста 394 символа, медиана при этом 205 символов - сказывается наличие выбросов (самый длинный комментарий - 5000 символов).
3. В данных имеет место дисбаланс классов - токсичных комментариев всего 10%
3. Мы сразу же отбросили все тексты с длиной более 512 символов, так как на следующих этапах планируем использовать модель BERT для токенизации и создания эмбеддингов. 
    * В результате потеряли около 20% данных.
    * Дисбаланс классов сохранился, 11% целевого класса.

### Downsampling

Для более качественного обучения моделей классификации выполним *dowwnsampling*, уменьшив количество нетоксичных текстов до количества токсичных (получим соотношение классов 1:1).

In [10]:
def downsample(X, y, fraction):
    X_zeros = X[y == 0]
    X_ones = X[y == 1]
    y_zeros = y[y == 0]
    y_ones = y[y == 1]

    X_downsampled = pd.concat(
        [X_zeros.sample(frac=fraction, random_state=RS)] + [X_ones])
    y_downsampled = pd.concat(
        [y_zeros.sample(frac=fraction, random_state=RS)] + [y_ones])
    
    X_downsampled, y_downsampled = shuffle(
        X_downsampled, y_downsampled, random_state=RS)
    
    return X_downsampled, y_downsampled

Доля остающихся нетоксичных комментариев 0.13 - выбрана таким образом, чтобы достичь соотношения классов 1:1.

In [11]:
X, y = downsample(X, y, 0.13)
X.shape, y.shape

((28797,), (28797,))

### Очистка признаков

Поскольку комментарии это сырой текст, набранный пользователями  в интернете, нам необходимо его немного почистить.

Функция `preprocess_text` принимает на вход текст комментария и возвращает очищенный комментарий:
* Удаляет все цифры и знаки пунктуации, оставляя только апостроф, т.к. он важен для английских слов;
* Удаляет одинокие символы (например, артикли);
* Удаляет множественные пробелы, которые могли появиться на предыдущих шагах очистки.

In [12]:
def preprocess_text(comment):
    # удаление цифр и пунктуации, кроме апострофа
    comment = re.sub("[^a-zA-Z']", " ", comment)

    # удаление одиноких символов
    comment = re.sub(r"\s+[a-zA-Z]\s+", ' ', comment)

    # удаление множественных пробелов
    comment = re.sub(r'\s+', ' ', comment)

    return comment

In [13]:
X = X.apply(preprocess_text)
X.apply(len).describe()

count    28797.000000
mean       155.189499
std        117.687178
min          1.000000
25%         59.000000
50%        121.000000
75%        226.000000
max        502.000000
Name: text, dtype: float64

**Вывод**

Наш корпус текстов выглядит уже намного лучше:
* Во-первых, их стало меньше - 28797 текстов;
* Во-вторых теперь они более сбалансированные по длине - среднее 155, медиана 121;
* В-третьих, очищены от ненужных символов.

### Токенизация текстов

Для токенизации наших комментариев, а также для создания эмбеддингов мы будем использовать модель BERT, специально обученную для задач классификации текстов по токсичности. Модель называется `toxic-bert` и создана командой **The Conversation AI**, исследовательской инициативой компаний Jigsaw и Google. Модель создавалась для поддержания здоровой атмосферы в онлайн-беседах и обучалась на корпусе английской википедии, а также на комментариях пользователей из интернета.

[Ссылка на репозиторий unitary/toxic-bert](https://huggingface.co/unitary/toxic-bert)

In [14]:
%%time
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')

tokenized = X.apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))
tokenized.shape

CPU times: total: 20.5 s
Wall time: 23 s


(28797,)

In [15]:
max_len = max(tokenized.apply(len))

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [16]:
padded.shape

(28797, 178)

### Создание эмбеддингов

Для создания эмбеддингов будем использовать весь наш сбалансированный датасет (28797 текстов).
Используем всё ту же предобученную на токсичных текстах модель.

In [17]:
model = AutoModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Так как эмбеддинги создаются довольно долго (около 3-х часов на домашней машине), для удобства ревьювинга архив с дампом эмбеддингов загружен на облачное хранилище, откуда и подгружается при наличии интернета и доступности ссылки.

В следующей ячейке этот подход реализован в виде ветвящегося кода:
* Если ссылка актвна (код ответа 200), забираем по ней npz-файл (85 Мб), это архив numpy, он имеет структуру словаря, поэтому для дальнейшей работы его необходимо распарсить в список массивов;
    * После загрузки файла проверяем количество батчей, количество эмбеддингов в батче и длину каждого эмбеддинга.
* Если ссылка недоступна (код ответа 404), тогда создаём эмбеддинги заново (143 батча по 200 текстов в каждом) и по завершении сохраняем их на диск в файл `embeddings_for_toxic_comments`
* В случае других ошибок возвращается сообщение с просьбой проверить интернет-соединение.

In [18]:
%%time

response = requests.get('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/2oUi72Fa6uBgsQ')

if response.status_code == 200:
    print("Npz-file with embeddings found, unpacking in progress.\n")
    embeddings_zip = np.load(BytesIO(response.content)) 
    embeddings = []
    for key, arr in embeddings_zip.items():
        embeddings.append(arr)
    print('Проверим размерность загруженного файла с эмбеддингами:')
    print("""
    Количество батчей: {0}
    Количество текстов в одном батче: {1}
    Длина эмбеддинга одного текста: {2}\n""".format(len(embeddings),
                                                  len(embeddings[0]),
                                                  len(embeddings[0][0])
                                                  )
         )
    
elif response.status_code == 404:
    print("The file was not found with embeddings by the link, proceed to create embeddings again.")
    batch_size = 200
    embeddings = []
    for batch_number in notebook.tqdm(range(padded.shape[0] // batch_size)):
            start = batch_size*batch_number
            end = batch_size*(batch_number + 1)

            batch = torch.LongTensor(padded[start:end]) 
            attention_mask_batch = torch.LongTensor(attention_mask[start:end])

            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            embeddings.append(batch_embeddings[0][:,0,:].numpy())

    np.savez("embeddings_for_toxic_comments", *embeddings)

else:
    print('Check your internet-connection!')

Npz-file with embeddings found, unpacking in progress.

Проверим размерность загруженного файла с эмбеддингами:

    Количество батчей: 143
    Количество текстов в одном батче: 200
    Длина эмбеддинга одного текста: 768

CPU times: total: 984 ms
Wall time: 7.69 s


### Разделение на выборки

При создании эмбеддингов мы брали тексты из выборки батчами по 200 штук, поэтому получилось некоторое расхождение в количестве наблюдений между X и y. Просто обрежем наши целевые метки **y** до длины вектора признаков. *Таким образом мы терям 197 наблюдений.*

In [19]:
X = np.concatenate(embeddings)
X.shape

(28600, 768)

In [20]:
y = y[:X.shape[0]].reset_index(drop=True).values
y.shape

(28600,)

Разделяем исходный набор на выборки, под тест отводим 20% данных.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RS)
X_train.shape, X_test.shape

((22880, 768), (5720, 768))

## Обучение

Обучение моделей будем проводить с помощью фреймворка **Optuna**, задача которого оптимальным образом подбирать гиперпараметры для любых моделей.

* Для контроля обобщающей способности моделей и переобучения используем кросс-валидацию с делением обучающей выборки на 5 фолдов;
* Результаты работы каждой модели сохраняем в датафрейм, индексами которого являются метрики F1 (среднее гармонической точности и полноты). Здесь сохраняем сразу и F1 с обучающей выборки (с кросс-валидации) и с тестовой выборки.

In [22]:
kf = KFold(n_splits=5, shuffle=True, random_state=RS)

In [23]:
results = pd.DataFrame(index=['f1_train', 'f1_test'])

Функция `optuna_search` принимает на вход функцию, которую необходимо оптимизировать, количество испытаний в исследовании и имя исследования. Внутри:
1. Создаем экземпляр класса `study` из модуля Optuna, задаём направление оптимизации функции (так как у нас F1, то её максимизируем для повышения качества модели);
2. Запускам собственно процесс оптимизации созданного исследования (`study`) с заданным количеством испытаний (`trial`).

In [24]:
def optuna_search(obj_func, n_trials, study_name='unnamed'):
    study = optuna.create_study(direction='maximize', study_name=study_name)
    study.optimize(obj_func, n_trials=n_trials)
    
    print(f"\tBest value (F1-score): {study.best_value:.5f}")
    print(f"\tBest params:")

    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")
    return study

Функция `metric_fixer` необходима для фиксации результатов в вышеописанный датафрейм `results`:
1. Принимает на вход оптимизированный объект класса `optuna.Study` и строковое имя модели, которую мы оптимизировали;
2. Из словаря берёт чистый объект модели (лог.регрессию, случайный лес или LightGBM-классификатор);
3. Инициализирует модель с параметрами, которые были выбраны как наилучшие в процессе исследования Оптуной;
4. Обучает модель на полной обучающей выборке;
5. Считает метрику качества F1 на тестовой выборке и записывает в датафрейм `results`. Туда же записывает и наилучшее значение F1, полученной Оптуной в процессе обучения.

In [25]:
def metric_fixer(study: optuna.Study, model):
    models_dict = {
        'lr': LogisticRegression(random_state=RS),
        'rf': RandomForestClassifier(),
        'lgbm': LGBMClassifier()
    } 
    
    model = models_dict.get(model).set_params(**study.best_params)
    model.fit(X_train, y_train)
    
    score_train = study.best_value
    score_test = f1_score(y_test, model.predict(X_test))
    
    results[type(model).__name__] = [score_train, score_test]
    return results

### Baseline

В качестве базовой точки для обучения, а также проверки на адекватность наших следующих моделей используем думми-классификатор из библиотеки `sklearn`.

In [26]:
dummy = DummyClassifier(strategy='stratified', random_state=RS)
dummy.fit(X_train, y_train)
pred = dummy.predict(X_test)
score_train = f1_score(y_train, dummy.predict(X_train))
score_test = f1_score(y_test, pred)
results['DummyClassifier'] = [score_train, score_test]
print('F1 метрика для думми-классификатора: {0}'.format(round(score_test, 5)))


F1 метрика для думми-классификатора: 0.47616


### LogisticRegression

In [27]:
def objective_lr(trial: optuna.Trial):
    logreg_c = trial.suggest_float('C', 1e-10, 1e10, log=True)
    tol = trial.suggest_float('tol', 1e-5, 1e-1, log=True)
    model = LogisticRegression(C=logreg_c)
    scores = cross_validate(model, X_train, y_train, cv=kf, scoring="f1")
    return np.mean(scores['test_score'])

In [29]:
%%time
lr_study = optuna_search(objective_lr, 10, 'LogReg')

[I 2022-06-17 14:12:29,494] A new study created in memory with name: LogReg
[I 2022-06-17 14:12:40,130] Trial 0 finished with value: 0.9730424048666219 and parameters: {'C': 1846.6599842808337, 'tol': 0.012417885322843202}. Best is trial 0 with value: 0.9730424048666219.
[I 2022-06-17 14:12:42,389] Trial 1 finished with value: 0.0 and parameters: {'C': 1.0521786231743486e-09, 'tol': 0.00082776632964173}. Best is trial 0 with value: 0.9730424048666219.
[I 2022-06-17 14:12:52,167] Trial 2 finished with value: 0.9735213611913253 and parameters: {'C': 1022633.7377555621, 'tol': 0.0002099004260738631}. Best is trial 2 with value: 0.9735213611913253.
[I 2022-06-17 14:12:57,565] Trial 3 finished with value: 0.9767756520585076 and parameters: {'C': 0.0012171590035575995, 'tol': 1.4106294375622711e-05}. Best is trial 3 with value: 0.9767756520585076.
[I 2022-06-17 14:13:07,929] Trial 4 finished with value: 0.9760801218441719 and parameters: {'C': 0.06914939976068218, 'tol': 0.000790211875885772

	Best value (F1-score): 0.97678
	Best params:
		C: 0.0012171590035575995
		tol: 1.4106294375622711e-05
CPU times: total: 8min 27s
Wall time: 1min 29s


In [31]:
%%time
metric_fixer(lr_study, 'lr')

CPU times: total: 7.02 s
Wall time: 1.23 s


,DummyClassifier,LogisticRegression
f1_train,0.495070,0.976776
f1_test,0.476157,0.969912


### Random Forest

In [32]:
def objective_rf(trial: optuna.Trial):
    params = {
    'random_state': RS,
    'n_estimators': trial.suggest_int('n_estimators', 10, 500),
    'max_depth': trial.suggest_int('max_depth', 1, 32, log=True),
    'max_features': trial.suggest_int('max_features', 3, 27),
#     'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 9, step=3),
#     'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
#     'criterion': trial.suggest_categorical('criterion', ["gini", "entropy"])
          }
    model = RandomForestClassifier(**params)
    scores = cross_validate(model, X_train, y_train, cv=kf, scoring="f1", n_jobs=-1)
    return np.mean(scores['test_score'])

In [69]:
%%time
rf_study = optuna_search(objective_rf, 10, 'RandomForest')

[I 2022-06-17 10:18:05,352] A new study created in memory with name: RandomForest
[I 2022-06-17 10:18:30,067] Trial 0 finished with value: 0.9735079457855292 and parameters: {'n_estimators': 318, 'max_depth': 1, 'max_features': 8}. Best is trial 0 with value: 0.9735079457855292.
[I 2022-06-17 10:19:04,407] Trial 1 finished with value: 0.9738434065437961 and parameters: {'n_estimators': 440, 'max_depth': 1, 'max_features': 8}. Best is trial 1 with value: 0.9738434065437961.
[I 2022-06-17 10:24:17,794] Trial 2 finished with value: 0.9764032500086544 and parameters: {'n_estimators': 308, 'max_depth': 8, 'max_features': 18}. Best is trial 2 with value: 0.9764032500086544.
[I 2022-06-17 10:32:17,662] Trial 3 finished with value: 0.9763235819106102 and parameters: {'n_estimators': 359, 'max_depth': 7, 'max_features': 27}. Best is trial 2 with value: 0.9764032500086544.
[I 2022-06-17 10:43:10,648] Trial 4 finished with value: 0.9760294305104832 and parameters: {'n_estimators': 254, 'max_depth

	Best value (F1-score): 0.97640
	Best params:
		n_estimators: 308
		max_depth: 8
		max_features: 18
CPU times: user 25min, sys: 93.9 ms, total: 25min
Wall time: 25min 5s


Сохранение результатов исследования на локальный диск для удобства продолжения оптимизации в случае перезагрузки ядра.

In [ ]:
# joblib.dump(rf_study, "rf_study.pkl")

Загрузка результатов из дампа на локальном диске.

In [33]:
# rf_study = joblib.load("rf_study.pkl")

In [34]:
%%time
metric_fixer(rf_study, 'rf')

CPU times: total: 1min 35s
Wall time: 1min 35s


,DummyClassifier,LogisticRegression,RandomForestClassifier
f1_train,0.495070,0.976776,0.976487
f1_test,0.476157,0.969912,0.969761


### LightGBMClassifier

In [35]:
def objective_lgbm(trial: optuna.Trial):
    params = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
#         "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
#         "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
#         "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
#         "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
#         "feature_fraction": trial.suggest_float(
#             "feature_fraction", 0.2, 0.95, step=0.1),
#         "max_bin": trial.suggest_int("max_bin", 30, 300)
    }
    model = LGBMClassifier(**params)
    scores = cross_validate(model, X_train, y_train, cv=kf, scoring="f1", n_jobs=None)
    return np.mean(scores['test_score'])

In [ ]:
%%time
lgbm_study = optuna_search(objective_lgbm, 10, 'LGBM')

[I 2022-06-17 10:45:21,330] A new study created in memory with name: LGBM


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=6700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6700
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5


In [ ]:
# joblib.dump(lgbm_study, "lgbm_study.pkl")

In [36]:
# lgbm_study = joblib.load("lgbm_study.pkl")

In [37]:
%%time
metric_fixer(lgbm_study, 'lgbm')

[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
CPU times: total: 2min 34s
Wall time: 22.4 s


,DummyClassifier,LogisticRegression,RandomForestClassifier,LGBMClassifier
f1_train,0.495070,0.976776,0.976487,0.976141
f1_test,0.476157,0.969912,0.969761,0.971530


## Анализ результатов

In [38]:
round(results, 5)

,DummyClassifier,LogisticRegression,RandomForestClassifier,LGBMClassifier
f1_train,0.49507,0.97678,0.97649,0.97614
f1_test,0.47616,0.96991,0.96976,0.97153


* Все модели показали себя очень хорошо, **метрика F1 не опускалась ниже 0.96** ни на обучающей, ни на тестовой выборке;
* Вероятнее всего это связано с хорошей подготовкой данных перед обучением, включая использование очень хорошей и специфичной к токсичности предобученной модели от команды **Conversation AI**;
* Учитывая скорость обучения и скорость инференса, мы рекомендуем заказчику использовать логистическую регрессию. Она уступает модели бустинга лишь в 3-м знаке после запятой, зато работает значительно быстрее;
* К сожалению, из-за специфики работы BERT'а невозможно после обучения визуализировать наиболее важные слова/фразы/предложения, на которые опирались модели в решении задачи классификации.

## Выводы

В этом проекте мы использовали датасет из 159 тысяч текстовых комментариев из магазина "Викишоп", размеченных по токсичности (1/0) чтобы обучить модель классификации. В процессе работы:
1. Ознакомились с данными и базовыми описательными статистиками;
2. В связи с дисбалансом классов (90/10) уменьшили количество нетоксичных текстов, случайно выбрав 13% из них, т.о. достигнув соотношения токсичных и нетоксичных текстов 50/50;
3. Перед токенизацией и векторизацией текстов выполнили очистку от знаков пунктуации, цифр, нелатинских букв, одиноких символов и множественных пробелов;
4. Токенизацию и векторизацию текстов выполнили с помощью предобученной на большом корпусе модифицированной BERT-модели (*unitary/toxic-bert* в репозитории HuggingFace). Эта модель была обучена при поддержке Google и высокоспецифична к выявлению токсичность в онлайн-беседах;
5. В данном проекте мы обучали классификации 3 модели из разных классов: логистическая регрессия, случайный лес и бустинг LightGBM. Все модели достигли прекрасных показателей качества с метрикой **F1 более 0.96 на тестовой выборке**.
6. Учитывая такие факторы, как скорость обучения и предсказания модели, а также её сложность, **мы рекомендуем заказчику ("Викишоп") использовать в данном случае логистическую регрессию**.